# IS319 - Deep Learning

## TP3 - Recurrent neural networks

Credits: Andrej Karpathy

The goal of this TP is to experiment with recurrent neural networks for a character-level language model to generate text that looks like training text data.

In [31]:
import torch
import torch.nn as nn
import torch.optim as optimizer
import numpy as np
import torch.nn.functional as F
import torch.distributions as distributions
import matplotlib.pyplot as plt
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available() # For macOS
    else "cpu"
)
print(f'Using {device}')

Using cpu


## 1. Text data preprocessing

Several text datasets are provided, feel free to experiment with different ones throughout the TP. At the beginning, use a small subset of a given dataset (for example use only 10k characters).

In [2]:
dir_datasets = "./"
# text_data_fname = 'baudelaire.txt'  # ~0.1m characters (French)
# text_data_fname = 'proust.txt'      # ~7.3m characters (French)
text_data_fname = 'shakespeare.txt' # ~0.1m characters (English)
# text_data_fname = 'lotr.txt'        # ~2.5m characters (English)
# text_data_fname = 'doom.c'          # ~1m characters (C Code)
# text_data_fname = 'linux.c'         # ~11.5m characters (C code)

text_data = open(dir_datasets+text_data_fname, 'r').read()
text_data = text_data[:10000] # use a small subset
print(f'Dataset `{text_data_fname}` contains {len(text_data)} characters.')
print('Excerpt of the dataset:')
print(text_data[:2000])

Dataset `shakespeare.txt` contains 10000 characters.
Excerpt of the dataset:
    SONNETS



TO THE ONLY BEGETTER OF
THESE INSUING SONNETS
MR. W. H. ALL HAPPINESS
AND THAT ETERNITY
PROMISED BY
OUR EVER-LIVING POET WISHETH
THE WELL-WISHING
ADVENTURER IN
SETTING FORTH
T. T.


I.

FROM fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou, contracted to thine own bright eyes,
Feed'st thy light'st flame with self-substantial fuel,
Making a famine where abundance lies,
Thyself thy foe, to thy sweet self too cruel.
Thou that art now the world's fresh ornament
And only herald to the gaudy spring,
Within thine own bud buriest thy content
And, tender churl, makest waste in niggarding.
  Pity the world, or else this glutton be,
  To eat the world's due, by the grave and thee.

II.

When forty winters shall beseige thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's p

**(Question)** Create a character-level vocabulary for your text data. Create two dictionaries: `ctoi` mapping each character to an index, and the reverse `itoc` mapping each index to its corresponding character. Implement the functions to convert text to tensor and tensor to text using these mappings. Apply these functions to some text data.

In [3]:
# Create the vocabulary and the two mapping dictionaries
def create_vocab(text):
    vocab, ctoi, itoc = [], {}, {}
    for character in text :
        if character not in vocab :
            vocab += [character]
            ctoi[character] = vocab.index(character)
            itoc[vocab.index(character)] = character
    return vocab, ctoi, itoc


# Implement the function converting text to tensor
def text_to_tensor(text, ctoi):
    arr = np.zeros((len(text)))
    for idx, char in enumerate(text):
        arr[idx] = ctoi[char]
    return torch.tensor(arr, dtype=torch.long).to(device)



# Implement the function converting tensor to text
def tensor_to_text(tensor, itoc):
    arr = tensor.cpu().detach().numpy()
    text = ""
    for elm in arr :
        text+=itoc[elm]
    return text

# Apply your functions to some text data

vocab, ctoi, itoc = create_vocab(text_data)

print(vocab)
print(ctoi)
print(itoc)

example_tensor = text_to_tensor(text_data, ctoi)
print(example_tensor)

example_text = tensor_to_text(example_tensor, itoc)
# verify integrity
assert example_text == text_data

[' ', 'S', 'O', 'N', 'E', 'T', '\n', 'H', 'L', 'Y', 'B', 'G', 'R', 'F', 'I', 'U', 'M', '.', 'W', 'A', 'P', 'D', 'V', '-', 'f', 'a', 'i', 'r', 'e', 's', 't', 'c', 'u', 'w', 'd', 'n', ',', 'h', 'b', 'y', "'", 'o', 'm', 'g', 'v', 'p', 'l', ':', 'k', 'z', 'x', '!', ';', '?', 'C', 'q', 'j', 'X']
{' ': 0, 'S': 1, 'O': 2, 'N': 3, 'E': 4, 'T': 5, '\n': 6, 'H': 7, 'L': 8, 'Y': 9, 'B': 10, 'G': 11, 'R': 12, 'F': 13, 'I': 14, 'U': 15, 'M': 16, '.': 17, 'W': 18, 'A': 19, 'P': 20, 'D': 21, 'V': 22, '-': 23, 'f': 24, 'a': 25, 'i': 26, 'r': 27, 'e': 28, 's': 29, 't': 30, 'c': 31, 'u': 32, 'w': 33, 'd': 34, 'n': 35, ',': 36, 'h': 37, 'b': 38, 'y': 39, "'": 40, 'o': 41, 'm': 42, 'g': 43, 'v': 44, 'p': 45, 'l': 46, ':': 47, 'k': 48, 'z': 49, 'x': 50, '!': 51, ';': 52, '?': 53, 'C': 54, 'q': 55, 'j': 56, 'X': 57}
{0: ' ', 1: 'S', 2: 'O', 3: 'N', 4: 'E', 5: 'T', 6: '\n', 7: 'H', 8: 'L', 9: 'Y', 10: 'B', 11: 'G', 12: 'R', 13: 'F', 14: 'I', 15: 'U', 16: 'M', 17: '.', 18: 'W', 19: 'A', 20: 'P', 21: 'D', 22: 

## 2. Setup a character-level recurrent neural network

**(Question)** Setup a simple embedding layer with `nn.Embedding` to project character indices to `embedding_dim` dimensional vectors. Explain precisely how this layer works and what are its outputs for a given input sequence.

In [4]:
# n_vocab : the total number of unique indices that the embedding layer can handle.
# n_dim : the size of the vector space in which the indices will be embedded.
n_vocab, n_dim = len(vocab), 10

# initiate the Embedding layer
emb_layer = nn.Embedding(n_vocab, embedding_dim=n_dim).to(device)

# given the example tensor generate an embedding of each index (indirectly character) of the text
emb_data = emb_layer(example_tensor)

print(emb_data)

tensor([[-0.5878, -0.4023,  0.5235,  ...,  1.7100,  0.2531, -0.4672],
        [-0.5878, -0.4023,  0.5235,  ...,  1.7100,  0.2531, -0.4672],
        [-0.5878, -0.4023,  0.5235,  ...,  1.7100,  0.2531, -0.4672],
        ...,
        [ 0.3893, -0.8002, -0.2066,  ..., -0.1047,  1.1033,  2.0073],
        [-0.7658,  1.2946, -1.1468,  ..., -1.7767,  2.5461,  0.0555],
        [ 0.3893, -0.8002, -0.2066,  ..., -0.1047,  1.1033,  2.0073]],
       grad_fn=<EmbeddingBackward0>)


**(Question)** Setup a single-layer RNN with `nn.RNN` (without defining a custom class). Use `hidden_dim` size for hidden states. Explain precisely the outputs of this layer for a given input sequence.

In [5]:
input_size, hidden_size = 10, 16
rnn_layer = nn.RNN(input_size, hidden_size).to(device)

# Initialize the hidden state
hidden_state = torch.zeros(1, hidden_size).to(device)

# run the embedded data through the RNN
output_sequence, final_hidden_state = rnn_layer(emb_data, hidden_state)

print(output_sequence.shape, output_sequence)
print(final_hidden_state)

torch.Size([10000, 16]) tensor([[ 0.2906, -0.5157,  0.5501,  ...,  0.3391, -0.1564,  0.1479],
        [ 0.3277, -0.4425,  0.4601,  ...,  0.3865, -0.1235,  0.4458],
        [ 0.3392, -0.4136,  0.5515,  ...,  0.4074, -0.2332,  0.2791],
        ...,
        [ 0.1011,  0.2952,  0.0370,  ...,  0.4565,  0.0609, -0.1029],
        [ 0.5068,  0.2476, -0.1606,  ..., -0.8468,  0.9422, -0.3244],
        [-0.2022, -0.1975,  0.4942,  ..., -0.2747,  0.4718, -0.7326]],
       grad_fn=<SqueezeBackward1>)
tensor([[-0.2022, -0.1975,  0.4942, -0.6849, -0.6139, -0.8608, -0.9111, -0.7187,
         -0.8841,  0.7647, -0.0874, -0.2705,  0.5366, -0.2747,  0.4718, -0.7326]],
       grad_fn=<SqueezeBackward1>)


## Answer :
The `output_sequence` represents the output of the RNN at each time step when it processes an input sequence. It is a tensor that contains the predicted value for each hidden state at each time step in the sequence. And the `final_hidden_state` is, as shown, the final hidden state of the layer.
****

**(Question)** Implement a simple training loop to overfit on a small input sequence. The loss function should be a categorical cross entropy on the predicted characters. Monitor the loss function value over the iterations.

In [24]:
# Sample a small input sequence into tensor `input_seq` and store its corresponding expected sequence into tensor `target_seq`
input_seq = torch.arange(0, 200).long().to(device)
target_seq = input_seq.clone().to(device)

# Implement a training loop overfitting an input sequence and monitoring the loss function
def train_overfit(model, input_seq, target_seq, optim, loss_function, n_iters=200, learning_rate=0.002, check_iter=50):

    for i in range(n_iters):
        # Forward pass
        output, hidden = model(input_seq.unsqueeze(0))  # Add batch dimension
        loss = loss_function(output.squeeze(0), target_seq)

        # Backward pass and optimization
        optim.zero_grad()
        loss.backward()
        optim.step()

        if (i + 1) % check_iter == 0:
            print(f"Iteration {i + 1}/{n_iters}, Loss: {loss.item()}")

# Initialize a model and make it overfit the input sequence
class simpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(simpleRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size).to(device)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True).to(device)
        self.fc = nn.Linear(hidden_size, output_size).to(device)

    def forward(self, x):
        x = self.embedding(x)
        output, hidden = self.rnn(x)
        output = self.fc(output)
        return output, hidden

# Initialize the model, loss function, and optimizer (Adam)
input_size, hidden_size, output_size, learning_rate = 200, 16, 200, 0.002
model = simpleRNN(input_size, hidden_size, output_size).to(device)
optim = optimizer.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss().to(device)

# train the model
train_overfit(model, input_seq, target_seq, optim, loss_function, n_iters=900)

Iteration 50/900, Loss: 4.256770610809326
Iteration 100/900, Loss: 2.87568998336792
Iteration 150/900, Loss: 1.7066768407821655
Iteration 200/900, Loss: 0.9677966237068176
Iteration 250/900, Loss: 0.5831930637359619
Iteration 300/900, Loss: 0.3865405321121216
Iteration 350/900, Loss: 0.27759045362472534
Iteration 400/900, Loss: 0.21128316223621368
Iteration 450/900, Loss: 0.16761454939842224
Iteration 500/900, Loss: 0.1370665282011032
Iteration 550/900, Loss: 0.11468390375375748
Iteration 600/900, Loss: 0.09769631177186966
Iteration 650/900, Loss: 0.08444388210773468
Iteration 700/900, Loss: 0.07387518137693405
Iteration 750/900, Loss: 0.06529081612825394
Iteration 800/900, Loss: 0.05820725858211517
Iteration 850/900, Loss: 0.05228080227971077
Iteration 900/900, Loss: 0.04726152494549751


**(Question)** Implement a `predict_argmax` method for your `RNN` model. Then, verify your overfitting: use some characters of your input sequence as context to predict the remaining ones. Experiment with the current model and analyze the results.

In [27]:
class CharRNN(simpleRNN):
    def predict_argmax(self, context_tensor, n_predictions):
        predictions, hidden = [], None

        # Use the context tensor to apply the forward pass for the context tensor
        for char_index in context_tensor:
            output, hidden = self.forward(char_index.unsqueeze(0).unsqueeze(0))

        # Predict the next n_predictions indices
        for _ in range(n_predictions):
            # Forward pass with the last index and hidden state
            output, hidden = self.forward(context_tensor[-1].unsqueeze(0).unsqueeze(0))

            # Get the index of the predicted index using argmax
            predicted_index = output.squeeze(0).argmax().item()
            predictions.append(predicted_index)

            # Update the context tensor with the new prediction
            context_tensor = torch.cat((context_tensor, torch.tensor([predicted_index]).to(device)))

        return predictions

# Initialize a model and make it overfit as above
# Then, verify your overfitting by predicting characters given some context
model = CharRNN(input_size, hidden_size, output_size).to(device)
optim = optimizer.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss().to(device)

train_overfit(model, input_seq, target_seq, optim, loss_function, n_iters=900)

Iteration 50/900, Loss: 4.276690483093262
Iteration 100/900, Loss: 2.8943259716033936
Iteration 150/900, Loss: 1.706049919128418
Iteration 200/900, Loss: 0.9533588886260986
Iteration 250/900, Loss: 0.5674981474876404
Iteration 300/900, Loss: 0.37381160259246826
Iteration 350/900, Loss: 0.26604777574539185
Iteration 400/900, Loss: 0.2003570944070816
Iteration 450/900, Loss: 0.15754812955856323
Iteration 500/900, Loss: 0.12801118195056915
Iteration 550/900, Loss: 0.10665475577116013
Iteration 600/900, Loss: 0.09060660004615784
Iteration 650/900, Loss: 0.07816838473081589
Iteration 700/900, Loss: 0.06828247755765915
Iteration 750/900, Loss: 0.06026414781808853
Iteration 800/900, Loss: 0.05365373194217682
Iteration 850/900, Loss: 0.04812970012426376
Iteration 900/900, Loss: 0.04345893859863281
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]) tensor([29, 30, 31])
tensor([29, 30, 31]) [29, 29, 29]


In [30]:
# predict 3 indices
print(target_seq[30:33], model.predict_argmax(input_seq[:30], n_predictions=3))

tensor([30, 31, 32]) [29, 29, 29]


Using the argmax function to predict the next character can yield a deterministic generator always predicting the same characters. Instead, it is common to predict the next character by sampling from the distribution of output predictions, adding some randomness into the generator.

**(Question)** Implement a `predict_proba` method for your `RNN` model. It should be very similar to `predict_argmax`, but instead of using argmax, it should randomly sample from the output predictions. To do that, you can use the `torch.distribution.Categorical` class and its `sample()` method. Verify that your method correctly added some randomness.

In [34]:
class CharRNN(CharRNN):
    def predict_proba(self, context_tensor, n_predictions):
        predictions, hidden = [], None

        # Use the context tensor to apply the forward pass for the context tensor
        for char_index in context_tensor:
            output, hidden = self.forward(char_index.unsqueeze(0).unsqueeze(0))

        # Predict the next n_predictions characters by sampling from the distribution
        for _ in range(n_predictions):
            # Forward pass with the last character and hidden state
            output, hidden = self.forward(context_tensor[-1].unsqueeze(0).unsqueeze(0))

            # Use Categorical distribution to sample from the predicted probabilities
            categorical_dist = distributions.Categorical(logits=output.squeeze(0))
            predicted_index = categorical_dist.sample().item()
            predictions.append(predicted_index)

            # Update the context tensor with the new prediction
            context_tensor = torch.cat((context_tensor, torch.tensor([predicted_index]).to(device)))

        return predictions

# Verify that your predictions are not deterministic anymore
model = CharRNN(input_size, hidden_size, output_size).to(device)
optim = optimizer.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss().to(device)

train_overfit(model, input_seq, target_seq, optim, loss_function, n_iters=900)

Iteration 50/900, Loss: 4.2722392082214355
Iteration 100/900, Loss: 2.8732962608337402
Iteration 150/900, Loss: 1.6952910423278809
Iteration 200/900, Loss: 0.9506286382675171
Iteration 250/900, Loss: 0.5644074082374573
Iteration 300/900, Loss: 0.36946171522140503
Iteration 350/900, Loss: 0.26259541511535645
Iteration 400/900, Loss: 0.19822251796722412
Iteration 450/900, Loss: 0.15631602704524994
Iteration 500/900, Loss: 0.12732800841331482
Iteration 550/900, Loss: 0.1062973141670227
Iteration 600/900, Loss: 0.09045511484146118
Iteration 650/900, Loss: 0.07815736532211304
Iteration 700/900, Loss: 0.06837471574544907
Iteration 750/900, Loss: 0.06043390929698944
Iteration 800/900, Loss: 0.053878478705883026
Iteration 850/900, Loss: 0.04838962107896805
Iteration 900/900, Loss: 0.04373795539140701


In [37]:
# predict 3 indices
print(target_seq[30:35], model.predict_proba(input_seq[:30], n_predictions=5))

tensor([30, 31, 32, 33, 34]) [29, 166, 166, 168, 168]


## 3. Train the RNN model on text data

**(Question)** Adapt your previous code to implement a proper training loop for a text dataset. To do so, we need to specify a sequence length `seq_len`, acting similarly to the batch size in classic neural networks. Then, you can either randomly sample sequences of length `seq_len` from the text dataset over `n_iters` iterations, or properly loop over the text dataset for `n_epochs` epochs (with a random starting point for each epoch to ensure different sequences), to make sure the whole dataset is seen by the model. Feel free to adjust training and model parameters empirically. Start with a small model and a small subset of the text dataset, then move on to larger experiments. Remember to use GPU if available.

In [ ]:
# Create the text dataset, compute its mappings and convert it to tensor
# YOUR CODE HERE
raise NotImplementedError()

# Initialize training parameters
# YOUR CODE HERE
raise NotImplementedError()

# Initialize a character-level RNN model
# YOUR CODE HERE
raise NotImplementedError()

# Setup the training loop
# Regularly record the loss and sample from the model to monitor what is happening
# YOUR CODE HERE
raise NotImplementedError()

**(Question)** From your trained model, play around with its predictions: start with a custom input sequence and ask the model to predict the rest. Analyze and comment your results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Answer :
****

## 4. Experiment with different RNN architectures

**(Question)** Experiment with different RNN architecures. Potential ideas are multi-layer RNNs, GRUs and LSTMs. All models can be extended to multi-layer using the `num_layers` parameter. Analyze and comment your results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Answer :
****